In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

from config import Config
config = Config.load()

### PROMPT

In [2]:
from utils import prompt_loader
prompt_loader.list_prompts()

2025-05-30 23:30:34,141 - utils.prompt_loader - INFO - Preloaded prompt: github_agent.j2
2025-05-30 23:30:34,146 - utils.prompt_loader - INFO - Preloaded prompt: news_format.j2
2025-05-30 23:30:34,150 - utils.prompt_loader - INFO - Preloaded prompt: search.j2
2025-05-30 23:30:34,155 - utils.prompt_loader - INFO - Preloaded prompt: summarise_agent.j2
2025-05-30 23:30:34,159 - utils.prompt_loader - INFO - Preloaded prompt: supervisor_agent.j2
2025-05-30 23:30:34,164 - utils.prompt_loader - INFO - Preloaded prompt: supervisor_agent_input.j2


['github_agent',
 'news_format',
 'search',
 'summarise_agent',
 'supervisor_agent',
 'supervisor_agent_input']

In [3]:
SEARCH_PROMPT = await prompt_loader.get_prompt("search").template.render_async()

### OPENAI

In [5]:
from utils import extract_json_from_text
client = OpenAI()
response = client.responses.create(
    model=config.tool.openai_search_tools.model,
    tools=[{"type": config.tool.openai_search_tools.type, 
            "search_context_size": config.tool.openai_search_tools.search_context_size},],
    input=SEARCH_PROMPT,    
)
search_result  = extract_json_from_text(response.output_text)

In [6]:
search_result

[{'title': 'Anthropic Releases Claude 4 with Autonomous Functionality',
  'source': 'Anthropic',
  'date': '2025-05-22',
  'url': 'https://en.wikipedia.org/wiki/Claude_%28language_model%29',
  'summary': 'Anthropic has unveiled Claude 4, featuring two models: Claude Opus 4 and Claude Sonnet 4. Notably, Claude 4 can operate autonomously for extended periods, enhancing its utility in complex tasks.'},
 {'title': 'Mistral AI Introduces Devstral, a Code-Focused AI Model',
  'source': 'Mistral AI',
  'date': '2025-05-21',
  'url': 'https://en.wikipedia.org/wiki/Mistral_AI',
  'summary': "Mistral AI announced Devstral, a new AI model optimized for coding tasks. Devstral outperforms other open models like Google's Gemma 3 27B and DeepSeek's V3 on coding benchmarks."}]

In [6]:
search_result_first = search_result[0]
SUMMARISE_PROMPT = await prompt_loader.get_prompt("summarise_news").\
template.render_async(title=search_result_first.get("title"), 
                source=search_result_first.get("source"), 
                date=search_result_first.get("date"), 
                summary=search_result_first.get("summary"),
                url_ref=search_result_first.get("url"))

In [7]:
with open("summarize_prompt.txt", "w") as f:
    f.write(SUMMARISE_PROMPT)

In [8]:
response = client.responses.create(
    model=config.openai_search_tools.model,
    tools=[{"type": config.openai_search_tools.type, 
            "search_context_size": config.openai_search_tools.search_context_size},],
    input=SUMMARISE_PROMPT,    
)

In [9]:
print(response.output_text)

```markdown
## 📰 Anthropic Releases Claude 4 with Autonomous Functionality

**Source**: Anthropic  
**Date**: 2025-05-22  
**URL**: [https://www.anthropic.com/claude-4-release](https://www.anthropic.com/claude-4-release)

---

### 🔹 What Happened

On May 22, 2025, Anthropic unveiled its latest AI models, Claude Opus 4 and Claude Sonnet 4, during its inaugural developer conference. These models are designed to handle complex tasks with sustained focus, with Opus 4 capable of autonomously coding for extended periods, reportedly up to seven hours. ([axios.com](https://www.axios.com/2025/05/22/anthropic-claude-version-4-ai-model?utm_source=openai))

### 🔹 Why It Matters

The release of Claude 4 signifies a substantial advancement in AI capabilities, particularly in autonomous coding and complex problem-solving. This development positions Anthropic as a strong competitor in the AI industry, challenging existing models like OpenAI's GPT-4.1 and Google's Gemini 2.5 Pro. ([itpro.com](https://w

In [10]:
with open("response.md", "w") as f:
    f.write(response.output_text)

### Google

In [7]:
from googleapiclient.discovery import build
import os

#  https://programmablesearchengine.google.com/
#  https://developers.google.com/custom-search/v1/overview


def perform_search(api_key, cse_id, query):
    try:
        # Build the service object
        service = build("customsearch", "v1", developerKey=api_key)

        # Make the search request
        # You can add more parameters like num=5 for number of results
        res = service.cse().list(
            q=query,
            cx=cse_id,
            # num=3 # Uncomment to get a specific number of results (e.g., 3)
        ).execute()

        # Extract and print the results
        if 'items' in res:
            print(f"Search results for: '{query}'\n")
            for i, item in enumerate(res['items'], 1):
                title = item.get('title')
                link = item.get('link')
                snippet = item.get('snippet')

                print(f"--- Result {i} ---")
                if title:
                    print(f"Title: {title}")
                if link:
                    print(f"Link: {link}")
                if snippet:
                    print(f"Snippet: {snippet}")
                print("-" * 20 + "\n")
        else:
            print("No results found.")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please check your API key, Search Engine ID, and API quota.")

In [8]:
# --- Configuration ---
API_KEY = os.getenv("GOOGLE_SEARCH_API_KEY")
SEARCH_ENGINE_ID = os.getenv("GOOGLE_SEARCH_ID")
SEARCH_QUERY = 'how to learn python'

perform_search(API_KEY, SEARCH_ENGINE_ID, SEARCH_QUERY)

An error occurred: <HttpError 403 when requesting https://customsearch.googleapis.com/customsearch/v1?q=how+to+learn+python&cx=GOOGLE_SEARCH_ID&alt=json returned "Method doesn't allow unregistered callers (callers without established identity). Please use API Key or other form of API consumer identity to call this API.". Details: "[{'message': "Method doesn't allow unregistered callers (callers without established identity). Please use API Key or other form of API consumer identity to call this API.", 'domain': 'global', 'reason': 'forbidden'}]">
Please check your API key, Search Engine ID, and API quota.
